# Fitting a Logistic Regression Model - Lab

## Introduction

In the last lesson you were given a broad overview of logistic regression. This included an introduction to two separate packages for creating logistic regression models. In this lab, you'll be investigating fitting logistic regressions with `statsmodels`. For your first foray into logistic regression, you are going to attempt to build a model that classifies whether an individual survived the [Titanic](https://www.kaggle.com/c/titanic/data) shipwreck or not (yes, it's a bit morbid).


## Objectives

In this lab you will: 

* Implement logistic regression with `statsmodels` 
* Interpret the statistical results associated with model parameters

## Import the data

Import the data stored in the file `'titanic.csv'` and print the first five rows of the DataFrame to check its contents. 

In [3]:
# Import the data
import pandas as pd
import statsmodels.api as sm


df = pd.read_csv("titanic.csv",index_col= 0)
df.head()


,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Define independent and target variables

Your target variable is in the column `'Survived'`. A `0` indicates that the passenger didn't survive the shipwreck. Print the total number of people who didn't survive the shipwreck. How many people survived?

In [5]:
# Total number of people who survived/didn't survive
#define the target variable
y = df["Survived"]
#calculate the number of people who didnt survive(survived == 0)
num_did_not_survive = y.value_counts()[0]
#calculate the number of people who survived
num_who_survived = y.value_counts()[1]
#display the results
print(f"number of people who did not survive: {num_did_not_survive}")
print(f"number of people who survived: {num_who_survived}")


number of people who did not survive: 549
number of people who survived: 342


Only consider the columns specified in `relevant_columns` when building your model. The next step is to create dummy variables from categorical variables. Remember to drop the first level for each categorical column and make sure all the values are of type `float`: 

In [6]:
# Create dummy variables
relevant_columns = ['Pclass', 'Age', 'SibSp', 'Fare', 'Sex', 'Embarked', 'Survived']
#filter the relevant columns
df_relevant = df[relevant_columns]
#create dummy variables for categorical columns dropping the first level
dummy_dataframe = pd.get_dummies(df_relevant,drop_first=True)

print(dummy_dataframe.shape)

(891, 8)


Did you notice above that the DataFrame contains missing values? To keep things simple, simply delete all rows with missing values. 

> NOTE: You can use the [`.dropna()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html) method to do this. 

In [7]:
# Drop missing rows
dummy_dataframe = dummy_dataframe.dropna()
print(dummy_dataframe.shape)

(714, 8)


In [8]:
dummy_dataframe.head()

,Pclass,Age,SibSp,Fare,Survived,Sex_male,Embarked_Q,Embarked_S
PassengerId,,,,,,,,
1,3,22.0,1,7.2500,0,1,0,1
2,1,38.0,1,71.2833,1,0,0,0
3,3,26.0,0,7.9250,1,0,0,1
4,1,35.0,1,53.1000,1,0,0,1
5,3,35.0,0,8.0500,0,1,0,1


In [9]:
dummy_dataframe.columns

Index(['Pclass', 'Age', 'SibSp', 'Fare', 'Survived', 'Sex_male', 'Embarked_Q',
       'Embarked_S'],
      dtype='object')

Finally, assign the independent variables to `X` and the target variable to `y`: 

In [10]:
# Split the data into X and y
y = dummy_dataframe["Survived"]#target variable
X = dummy_dataframe.drop(columns="Survived",axis=1)#independent features/variables

In [11]:
y

PassengerId
1      0
2      1
3      1
4      1
5      0
      ..
886    0
887    0
888    1
890    1
891    0
Name: Survived, Length: 714, dtype: int64

In [12]:
X

,Pclass,Age,SibSp,Fare,Sex_male,Embarked_Q,Embarked_S
PassengerId,,,,,,,
1,3,22.0,1,7.2500,1,0,1
2,1,38.0,1,71.2833,0,0,0
3,3,26.0,0,7.9250,0,0,1
4,1,35.0,1,53.1000,0,0,1
5,3,35.0,0,8.0500,1,0,1
...,...,...,...,...,...,...,...
886,3,39.0,0,29.1250,0,1,0
887,2,27.0,0,13.0000,1,0,1
888,1,19.0,0,30.0000,0,0,1


## Fit the model

Now with everything in place, you can build a logistic regression model using `statsmodels` (make sure you create an intercept term as we showed in the previous lesson).  

> Warning: Did you receive an error of the form "LinAlgError: Singular matrix"? This means that `statsmodels` was unable to fit the model due to certain linear algebra computational problems. Specifically, the matrix was not invertible due to not being full rank. In other words, there was a lot of redundant, superfluous data. Try removing some features from the model and running it again.

In [13]:
# Build a logistic regression model using statsmodels
#add an intercept to the independent variable
X = sm.add_constant(X)
#fit the model
model = sm.Logit(y,X)
result = model.fit()


Optimization terminated successfully.
         Current function value: 0.443267
         Iterations 6


## Analyze results

Generate the summary table for your model. Then, comment on the p-values associated with the various features you chose.

In [14]:
# Summary table
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      706
Method:                           MLE   Df Model:                            7
Date:                Fri, 23 Aug 2024   Pseudo R-squ.:                  0.3437
Time:                        12:33:50   Log-Likelihood:                -316.49
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 1.103e-67
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          5.6503      0.633      8.921      0.000       4.409       6.892
Pclass        -1.2118      0.163     -7.433      0.000      -1.531      -0.892
Age           -0.0431      0.008     -5.250      0.0

In [15]:
result.params

const         5.650279
Pclass       -1.211771
Age          -0.043139
SibSp        -0.380550
Fare          0.001153
Sex_male     -2.623621
Embarked_Q   -0.825996
Embarked_S   -0.413010
dtype: float64

In [16]:
# Your comments here
""" A constant value of 5.650279 represents the log-odds of survival when all predictor variables are set to zero.
    pclass of -1.211771 this coefficient suggests that for each one unit increase in passenger class the chance of survival decreases by approximately -1.211.
""" 


' A constant value of 5.650279 represents the log-odds of survival when all predictor variables are set to zero.\n    pclass of -1.211771 this coefficient suggests that for each one unit increase in passenger class the chance of survival decreases by approximately -1.211.\n'

## Level up (Optional)

Create a new model, this time only using those features you determined were influential based on your analysis of the results above. How does this model perform?

In [18]:
# Your code here
#influential features are those with larger absolute values for their coefficients.
X = dummy_dataframe[["Sex_male","Pclass","Embarked_Q","SibSp","Embarked_S"]]
y = dummy_dataframe["Survived"]

#fit the model
model2 = sm.Logit(y,sm.add_constant(X))
result = model2.fit()


Optimization terminated successfully.
         Current function value: 0.464916
         Iterations 6


In [19]:
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:               Survived   No. Observations:                  714
Model:                          Logit   Df Residuals:                      708
Method:                           MLE   Df Model:                            5
Date:                Fri, 23 Aug 2024   Pseudo R-squ.:                  0.3117
Time:                        12:45:59   Log-Likelihood:                -331.95
converged:                       True   LL-Null:                       -482.26
Covariance Type:            nonrobust   LLR p-value:                 7.385e-63
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.7596      0.368     10.216      0.000       3.038       4.481
Sex_male      -2.6386      0.210    -12.557      0.000      -3.050      -2.227
Pclass        -0.9063      0.122     -7.413      0.0

In [ ]:
# Your comments here

## Summary 

Well done! In this lab, you practiced using `statsmodels` to build a logistic regression model. You then interpreted the results, building upon your previous stats knowledge, similar to linear regression. Continue on to take a look at building logistic regression models in Scikit-learn!